In [ ]:

from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import glob
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import os
from itertools import chain
from datetime import datetime, timedelta


urls = ['https://www.reddit.com/r/london/comments/15rqct3/ideas_for_london_activities_for_my_dad/',
'https://www.reddit.com/r/london/comments/rvordl/places_you_actively_avoid_in_london/',
 'https://www.reddit.com/r/london/comments/pdsnpm/what_do_people_actually_do_in_london_that_isnt/',
 'https://www.reddit.com/r/TravelHacks/comments/11hfoix/going_to_london_england_for_a_week_any/',
'https://www.reddit.com/r/travel/comments/14sc0t3/london_definitely_exceeded_my_expectations/'
'https://www.reddit.com/r/finedining/comments/13yc8vl/what_is_your_favorite_restaurant_in_london_in_2023/'
 'https://www.reddit.com/r/TravelHacks/comments/14ae4af/what_are_some_interesting_things_to_do_in_london/',
  'https://www.reddit.com/r/londonontario/comments/wm01m6/what_are_the_top_5_things_to_do_in_london/'
       'https://www.reddit.com/r/london/comments/1855c15/what_is_getting_better_in_london/'
]

the_dates = []
the_contents = []
def scrape(url):
    service = Service(executable_path=r'path/to/chromedriver.exe')
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--blink-settings=imagesEnabled=false')
    #options.add_argument("--headless")
    #options.add_argument("--no-sandbox")
    #options.add_argument("--window-size=1920,1080")


    driver = webdriver.Chrome(service=service,options=options)

    driver.get(url)

    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH,"//span/span[@class='flex items-center gap-xs']")))
    driver.execute_script("window.stop();")

    elements = driver.find_elements(By.XPATH, "//button[@class='text-tone-2 text-12 no-underline hover:underline px-xs py-xs flex ml-[3px] xs:ml-0 !bg-transparent !border-0']")
    #elements = driver.find_elements(By.LINK_TEXT, "more replies")
  
    time.sleep(30)
    # Click each element
    for element in elements:
        try:
            element.click()
        except Exception as e:
            print(f"Error clicking element: {e}")
            # Continue processing other elements
            
    time.sleep(30)
    # Click each element
    for element in elements:
        try:
            element.click()
        except Exception as e:
            print(f"Error clicking element: {e}")
            # Continue processing other elements
            


    time.sleep(10)
    print('one')
    dates =driver.find_elements(By.XPATH,"//faceplate-timeago[@class='text-neutral-content-weak text-12']/time")
    date = [con.get_attribute('title') for con in dates]
    print(len(dates))
    the_dates.append(date)

    print('two')
    contents =driver.find_elements(By.CLASS_NAME, "py-0.mx-sm.inline-block.max-w-full")
    content = [con.get_attribute('innerText') for con in contents]
    the_contents.append(content)
    #driver.quit()

    
for i in urls:
    scrape(i)
    print(i)
    print(len(the_dates))
    print(len(the_contents))

import itertools
da = list(itertools.chain(*the_dates))
co = list(itertools.chain(*the_contents))
data = {'date_posted': da, 'content': co}
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()

df.to_csv('london.csv')
all_files = glob.glob('*.csv')
df = pd.DataFrame()

# Read each CSV file and append it to the DataFrame
for file in all_files:
    data = pd.read_csv(file)
    df = df.append(data, ignore_index=True)

df.drop_duplicates(inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)

df.to_csv('london_reddit_merged_data.csv', index=False)